In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np


df = pd.read_csv('/content/suratnew.csv')
df.head()
# Assuming 'df' is your DataFrame containing the dataset



,name,datetime,Humidity,Pressure,Rain,Temperature
0,surat,01-01-2021,61.9,1012.3,0,23.7
1,surat,02-01-2021,64.2,1013.0,0,23.8
2,surat,03-01-2021,51.0,1012.1,0,22.6
3,surat,04-01-2021,53.4,1011.8,0,23.0
4,surat,05-01-2021,53.7,1013.3,0,22.5


In [ ]:
print(df.columns)

Index(['name', 'datetime', 'Humidity', 'Pressure', 'Rain', 'Temperature'], dtype='object')


In [ ]:
#columns_to_drop = ['dew', 'precipprob', 'winddir', 'solarradiation', 'windspeed']
#df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
df


,name,datetime,Humidity,Pressure,Rain,Temperature
0,surat,01-01-2021,61.9,1012.3,0,23.7
1,surat,02-01-2021,64.2,1013.0,0,23.8
2,surat,03-01-2021,51.0,1012.1,0,22.6
3,surat,04-01-2021,53.4,1011.8,0,23.0
4,surat,05-01-2021,53.7,1013.3,0,22.5
...,...,...,...,...,...,...
1064,surat,01-12-2023,75.0,1011.9,0,24.7
1065,surat,02-12-2023,63.4,1010.9,0,26.5
1066,surat,03-12-2023,65.2,1011.5,1,26.8
1067,surat,04-12-2023,64.8,1011.7,0,25.9


In [ ]:
df["target1"] = df.shift(-1)["Humidity"]
df["target2"] = df.shift(-1)["Pressure"]
df["target3"] = df.shift(-1)["Rain"]
df["target4"] = df.shift(-1)["Temperature"]

In [ ]:
df=df.iloc[:-1,:].copy()

In [ ]:
import pandas as pd

# Assuming 'datetime_column' is the name of your datetime column in DataFrame 'df'
df['datetime'] = pd.to_datetime(df['datetime'])

# Convert datetime to timestamp (seconds since the epoch)
df['timestamp'] = df['datetime'].astype(int) // 10**9  # Convert nanoseconds to seconds

# 'timestamp' column now contains the datetime column converted to a numerical format
df['timestamp'] = df['timestamp'].astype(float)


<ipython-input-28-a90f9d74e8a1>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['datetime'] = pd.to_datetime(df['datetime'])


In [ ]:

print(df.dtypes)

name                   object
datetime       datetime64[ns]
Humidity              float64
Pressure              float64
Rain                    int64
Temperature           float64
target1               float64
target2               float64
target3               float64
target4               float64
timestamp             float64
dtype: object


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame containing NaN values in columns 'column1', 'column2', 'column3', 'column4'
columns_with_nan = ['Humidity', 'Pressure','Rain','Temperature','target1', 'target2', 'target3', 'target4']

# Remove rows with NaN values in specific columns
df.dropna(subset=columns_with_nan, inplace=True)


In [ ]:
 #Define features (X) and targets (y)
X = df[['Humidity', 'Pressure','Rain','Temperature']]  # Features
y = df[['target1', 'target2', 'target3','target4']]  # Targets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Initialize the MultiOutputRegressor with Random Forest as base estimator
model = MultiOutputRegressor(rf_regressor)

# Fit the multi-output regression model
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model using metrics like Mean Squared Error and R-squared
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

Mean Squared Error: 7.475978758177583
R-squared Score: 0.8831753110916734


In [ ]:
!pip install firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd



# Access the database reference at the appropriate path within your Firebase Realtime Database
ref = db.reference('/Sensor')  # Replace with your data path

# Retrieve the data from Firebase into a Pandas DataFrame
data_snapshot = ref.get()
data_df = pd.DataFrame.from_dict(data_snapshot, orient='index')  # Assuming data is in dictionary format

# Display the first few rows of the DataFrame to understand its structure
print(data_df.head())


                      0
Humidity       50.50000
Pressure     1009.54602
Rain            0.00000
Temperature    26.60000


In [ ]:
array_from_df = data_df.values
print(array_from_df)


[[  50.5    ]
 [1009.54602]
 [   0.     ]
 [  26.6    ]]


In [ ]:
reshaped_array = array_from_df.reshape(1, -1, 4)[:, 0, :]
# Make predictions using the trained model
predictions_new = model.predict(reshaped_array)

formatted_predictions_new = [[round(val, 4) for val in pred] for pred in predictions_new]

# Display or use the predictions
print("Predicted values for Humidity, Pressure, Rainfall, Temperature:")
print(formatted_predictions_new)


Predicted values for Humidity, Pressure, Rainfall, Temperature:
[[52.998, 1009.772, 0.01, 27.122]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
data_type = type(formatted_predictions_new)
print(data_type)

<class 'list'>


In [ ]:
import numpy as np
# Extract the 'Rainfall' predictions from formatted_predictions_new
rain_values = [pred[2] for pred in formatted_predictions_new]

# Convert to NumPy array
rain_values_array = np.array(rain_values)

# Applying if-else condition to rain values based on the threshold
threshold = 0.5
rain_values_array = np.where(rain_values_array > threshold, 1, 0)

# Update the rain values in formatted_predictions_new
for i in range(len(formatted_predictions_new)):
    formatted_predictions_new[i][2] = rain_values_array[i]

print("Predicted values of Humidity, Pressure, Rainfall, Temperature for next day :")
print(formatted_predictions_new)

Predicted values of Humidity, Pressure, Rainfall, Temperature for next day :
[[52.998, 1009.772, 0, 27.122]]


In [ ]:
index_to_name = {
    0: 'Humidity',
    1: 'Pressure',
    2: 'Rainfall',
    3: 'Temperature'
}
# Convert floating-point values to strings in the dictionary
predicted_values = {
    index_to_name[i]: str(val) for i, val in enumerate(formatted_predictions_new[0])
}

# Print the predicted values with their corresponding names
print("Predicted values of Humidity, Pressure, Rainfall, Temperature for next day:")
for name, value in predicted_values.items():
    print(f"{name}: {value}")

# Reference to your Firebase Realtime Database
ref = db.reference('/ValuesPredicted')  # Replace with your desired database reference

# Push data to Firebase Realtime Database
ref.set({'predictions': predicted_values})

Predicted values of Humidity, Pressure, Rainfall, Temperature for next day:
Humidity: 52.998
Pressure: 1009.772
Rainfall: 0
Temperature: 27.122
